<a href="https://colab.research.google.com/github/HelenNunez/BALG_Project3/blob/master/Reddit_Cleanup_SR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [ ]:

# Run this and follow the instructions:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!cp /content/drive/My\ Drive/BALG_GoogleDriveLN/data -r /content/

In [ ]:
Reddit_Data = "data/reddit_election2020.csv"

In [ ]:
Reddit_Data = pd.read_csv(Reddit_Data)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,13,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
Reddit_Data

,Unnamed: 0,all_awardings,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,author_patreon_flair,author_premium,awarders,body,collapsed_because_crowd_control,created_utc,edited,gildings,id,is_submitter,link_id,locked,no_follow,parent_id,permalink,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,total_awards_received,treatment_tags,created,distinguished,author_cakeday,top_awarded_type,media_metadata
0,0,[],NaN,maybeathrowawayac,NaN,NaN,[],NaN,1∆,dark,text,t2_21f01q80,False,False,[],&gt;But I don’t think the actions of a candida...,NaN,1586415573,1.586416e+09,{},fmvfwjx,True,t3_fxlph7,False,True,t1_fmvd33o,/r/changemyview/comments/fxlph7/cmv_bernie_san...,1586423958,1,True,False,changemyview,t5_2w2s8,0,[],1.586441e+09,NaN,NaN,NaN,NaN
1,1,[],NaN,staedtler2018,NaN,NaN,[],NaN,NaN,NaN,text,t2_4syrut,False,False,[],"Biden is going to win, in all likelihood. All ...",NaN,1586415520,1.586416e+09,{},fmvful9,False,t3_fxe6sy,False,True,t3_fxe6sy,/r/politics/comments/fxe6sy/joe_biden_leads_do...,1586423932,1,True,False,politics,t5_2cneq,0,[],1.586441e+09,NaN,NaN,NaN,NaN
2,2,[],NaN,parhwy,NaN,NaN,[],NaN,NaN,NaN,text,t2_1u09pov,False,False,[],This is myopic. Sanders was *the* saviour? Sor...,NaN,1586415451,1.586416e+09,{},fmvfs2b,False,t3_fxndcy,False,True,t3_fxndcy,/r/collapse/comments/fxndcy/the_survival_of_th...,1586423898,2,True,False,collapse,t5_2qhw9,0,[],1.586441e+09,NaN,NaN,NaN,NaN
3,3,[],NaN,ethics_in_disco,NaN,NaN,[],NaN,NaN,NaN,text,t2_nne41,False,False,[],[Here's an exit poll by NYT](http://www.nytime...,NaN,1586415153,NaN,{},fmvfgx0,False,t3_fxg6lo,False,True,t1_fmunji1,/r/politics/comments/fxg6lo/return_to_normalcy...,1586423750,1,True,False,politics,t5_2cneq,0,[],1.586440e+09,NaN,NaN,NaN,NaN
4,4,[],NaN,CraggerJack1775,transparent,NaN,"[{'e': 'text', 't': ' '}]",63870ff8-4455-11ea-8218-0eb38c745dd7,,dark,richtext,t2_38n774vs,False,False,[],I was raised to vote red and thought that ever...,NaN,1586415112,NaN,{},fmvffet,False,t3_fx8w7b,False,True,t3_fx8w7b,/r/PresidentialRaceMemes/comments/fx8w7b/it_is...,1586423730,1,True,False,PresidentialRaceMemes,t5_17rom5,0,[],1.586440e+09,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106862,106862,[],NaN,3-10,NaN,NaN,[],91b486ac-8063-11e2-a885-12313d1839b0,Constitutional Paratrooper,dark,text,t2_12wezj9,False,True,[],Very well. \n\nActually none are insignificant...,NaN,1597734600,NaN,{},g1yjlg6,False,t3_ibftfv,False,True,t1_g1yiz1i,/r/Republican/comments/ibftfv/he_wasnt_wrong/g...,1597740870,1,True,False,Republican,t5_2qndt,0,[],1.597760e+09,NaN,NaN,NaN,NaN
106863,106863,[],NaN,AutoModerator,#dadada,NaN,"[{'e': 'text', 't': '🤖 RUS'}]",b5bb55a4-4cf2-11ea-a71f-0eaa6a3f3be5,🤖 RUS,dark,richtext,t2_6l4z3,False,True,[],\nThis subreddit is a pro-Trump subreddit for ...,NaN,1597734230,NaN,{},g1yj784,False,t3_ibw7xt,False,True,t3_ibw7xt,/r/trump/comments/ibw7xt/trump_says_to_pardon_...,1597740682,1,False,False,trump,t5_2r18q,0,[],1.597759e+09,moderator,NaN,NaN,NaN
106864,106864,[],NaN,autotldr,NaN,NaN,[],NaN,NaN,NaN,text,t2_6sklr,False,False,[],"This is the best tl;dr I could make, [original...",NaN,1597734126,NaN,{},g1yj36a,False,t3_ibmkeo,False,True,t3_ibmkeo,/r/Worldnews_Headline/comments/ibmkeo/trump_sa...,1597740632,1,True,False,Worldnews_Headline,t5_2holw9,0,[],1.597759e+09,NaN,NaN,NaN,NaN
106865,106865,[],NaN,autotldr,NaN,NaN,[],NaN,BOT,dark,text,t2_6sklr,False,False,[],"This is the best tl;dr I could make, [original...",NaN,1597734123,NaN,{},g1yj32d,False,t3_ibvzbn,False,True,t3_ibvzbn,/r/worldnews/comments/ibvzbn/us_watching_terri...,1597740630,3,True,False,worldnews,t5_2qh13,0,[],1.597759e+09,NaN,NaN,NaN,NaN


In [ ]:
list(Reddit_Data.columns.values) 

['Unnamed: 0',
 'all_awardings',
 'associated_award',
 'author',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_template_id',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'body',
 'collapsed_because_crowd_control',
 'created_utc',
 'edited',
 'gildings',
 'id',
 'is_submitter',
 'link_id',
 'locked',
 'no_follow',
 'parent_id',
 'permalink',
 'retrieved_on',
 'score',
 'send_replies',
 'stickied',
 'subreddit',
 'subreddit_id',
 'total_awards_received',
 'treatment_tags',
 'created',
 'distinguished',
 'author_cakeday',
 'top_awarded_type',
 'media_metadata']

In [ ]:
New_Data = Reddit_Data[["body", "subreddit", "retrieved_on"]]

In [ ]:
New_Data = New_Data.rename(columns={"body": "Text", "subreddit":"Subreddit", "retrieved_on":"Date"})
New_Data

,Text,Subreddit,Date
0,&gt;But I don’t think the actions of a candida...,changemyview,1586423958
1,"Biden is going to win, in all likelihood. All ...",politics,1586423932
2,This is myopic. Sanders was *the* saviour? Sor...,collapse,1586423898
3,[Here's an exit poll by NYT](http://www.nytime...,politics,1586423750
4,I was raised to vote red and thought that ever...,PresidentialRaceMemes,1586423730
...,...,...,...
106862,Very well. \n\nActually none are insignificant...,Republican,1597740870
106863,\nThis subreddit is a pro-Trump subreddit for ...,trump,1597740682
106864,"This is the best tl;dr I could make, [original...",Worldnews_Headline,1597740632
106865,"This is the best tl;dr I could make, [original...",worldnews,1597740630


In [ ]:
import datetime as datetime
New_Data['Date'] = New_Data['Date'].apply(lambda x:datetime.datetime.fromtimestamp(x))  
#New_Data['Date']

0        2020-04-09 09:19:18
1        2020-04-09 09:18:52
2        2020-04-09 09:18:18
3        2020-04-09 09:15:50
4        2020-04-09 09:15:30
                 ...        
106862   2020-08-18 08:54:30
106863   2020-08-18 08:51:22
106864   2020-08-18 08:50:32
106865   2020-08-18 08:50:30
106866   2020-08-18 08:50:29
Name: Date, Length: 106867, dtype: datetime64[ns]

In [ ]:
New_Data

,Text,Subreddit,Date
0,&gt;But I don’t think the actions of a candida...,changemyview,2020-04-09 09:19:18
1,"Biden is going to win, in all likelihood. All ...",politics,2020-04-09 09:18:52
2,This is myopic. Sanders was *the* saviour? Sor...,collapse,2020-04-09 09:18:18
3,[Here's an exit poll by NYT](http://www.nytime...,politics,2020-04-09 09:15:50
4,I was raised to vote red and thought that ever...,PresidentialRaceMemes,2020-04-09 09:15:30
...,...,...,...
106862,Very well. \n\nActually none are insignificant...,Republican,2020-08-18 08:54:30
106863,\nThis subreddit is a pro-Trump subreddit for ...,trump,2020-08-18 08:51:22
106864,"This is the best tl;dr I could make, [original...",Worldnews_Headline,2020-08-18 08:50:32
106865,"This is the best tl;dr I could make, [original...",worldnews,2020-08-18 08:50:30


In [ ]:
import nltk
nltk.download('punkt')

nltk.download('stopwords')
from nltk.corpus import stopwords

from pyspark.mllib.feature import Word2Vec


In [ ]:
sentences_list = []
Reddit_list = []
stop_words = set(stopwords.words("english"))
punctuations = [".", ",", "!", "?", ";", "#", ":", "(", ")", "-", "–" "/", "http", "https", "www", "\\", "@", "&", "[", "]","~"]

for article in New_Data:
    sentences = nltk.sent_tokenize(New_Data['text'].numpy().decode('utf-8'))
    sentences_list.append(sentences)
    for sentence in sentences:
      words = nltk.word_tokenize(sentence)
      without_stop_words = [
      word for word in words if word not in stop_words]
      clean_words = [
                     word for word in without_stop_words if word not in punctuations]
      #word for word in words if word not in stop_words or any(punc in word for punc in punctuations)]

      Reddit_list.append(clean_words)


In [ ]:
Reddit_list

In [ ]:
#Tokenize "Text" column
#Add stopwords

In [ ]:
# Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-ba

In [ ]:
from pyspark.mllib.feature import Word2Vec
word2vec = Word2Vec()

In [ ]:
# import saved model

In [ ]:
#Test model against Reddit API data
#Not sure how to do this. Should we model the Reddit data, then compare models? this doesn't make a whole lot of sense.
#Or should I set reddit data to y in the consine similarity?

synonyms = model.findSynonyms('man', 10)

for word, cosine_distance in synonyms:
     print("{}: {}".format(word, cosine_distance))

In [ ]:
def cosine_similarity(x, y):
     return x.dot(y)/(x.norm(2)*y.norm(2))

In [ ]:
x = model.transform('man')
y = model.transform('woman')
cosine_similarity(x, y)